<a href="https://colab.research.google.com/github/jdmonmou/W207-project/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Injection Detection Challenge

Step 1: Load the data

In [104]:
!pip install wget
import pandas as pd
import msgpack
import requests
import wget
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import PCA



  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=01c72227b9ca39647c7b014ed0ea454305fbe1fbf49ad729d08448ce474d648a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [105]:
#store paths to data
train_msgpack_url = "https://github.com/jdmonmou/W207-project/blob/master/train.msgpack?raw=true"
test_msgpack_url = "https://github.com/jdmonmou/W207-project/blob/master/test.msgpack?raw=true"
train_infocsv_url = "https://raw.githubusercontent.com/jdmonmou/W207-project/master/train_info.csv"

info = pd.read_csv(train_infocsv_url)
info.head()


,id,injection
0,14577,False
1,60186,True
2,58459,True
3,87091,True
4,44461,False


In [96]:
info.injection.value_counts()
#total size is 65,854
#almost even split between true and false classes

False    33499
True     32355
Name: injection, dtype: int64

In [117]:
!wget https://github.com/jdmonmou/W207-project/blob/master/train.msgpack?raw=true
train = msgpack.unpack(train.msgpack)
'''
r = wget.download(test_msgpack_url)
f = open(r, "r+")
test = msgpack.unpack(f)


train = pd.DataFrame(train)
test = pd.DataFrame(test)
train.columns = ["id","text"]
test.columns = ["id","text"]
'''

--2019-11-22 21:15:06--  https://github.com/jdmonmou/W207-project/blob/master/train.msgpack?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jdmonmou/W207-project/raw/master/train.msgpack [following]
--2019-11-22 21:15:06--  https://github.com/jdmonmou/W207-project/raw/master/train.msgpack
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jdmonmou/W207-project/master/train.msgpack [following]
--2019-11-22 21:15:06--  https://raw.githubusercontent.com/jdmonmou/W207-project/master/train.msgpack
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting resp

AttributeError: ignored

In [0]:
train.head(10).values

array([[59290,
        b"' and/**/38>( select\t(622)/**/from/*362 emhgpv gpnqdn odpxkn qgnoyb */htipaw)"],
       [54992,
        b'shqpkt" union /*!426 all\t*/(select kwicwt(\t(\tyaaoda\t()), (236), (konjlq()),  619, nyknas ()), byamlo (), \txfsesf( (\tdqoyrp()), (sbecbl()), (\tlhnjpg()), (kkicjp()), (udkygv()), hoewxv\t()))'],
       [64287,
        b'nhnqag" uniondistinct--154 298 plhlre exaloq unyote \r(select (rpfeir#15 256 15 rttlat 66 gsjxjq 44 819 \r()), kuoopt(), (#\rvxhuha), (116), (--kfbvww 193 \r"mkhkjq"), 271, (omqjmj), gvdira--\r((--\r325), --mcrgew oqrhwp jhseux 55 xpayss 88 \rfctnjs (), (794), 3, #58 14 114 vdqoyq \r\'apxkvx\', (\tmhjblm())), #yxnynl \r"lllpue"/**/from#\rexample.example)'],
       [28821, b'D8j+oNbylTIGw='],
       [27825,
        b'ened \\">\\n  <head>\\n    <script data-react-helmet=\\"true\\" type=\\"text/javascript\\">(function(w...<some bytes>....getTime(),event:\'example.example\'});var f=d.getElementsByTag...<some bytes>...re(j,f);\\n    })(windo

In [0]:
train["text"] = train["text"].astype(str)
test["text"] = test["text"].astype(str)
vectorizer = TfidfVectorizer(ngram_range=(1,4), analyzer='char')
full_text = list(train["text"].values) + list(test["text"].values)
vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train['text'])
test_vectorized = vectorizer.transform(test["text"])
info = pd.merge(train, info, on="id")
y = np.array([1 if i == True else 0 for i in info.injection.values])
logreg = LogisticRegression(C=10.0)
scores = cross_val_score(logreg, train_vectorized, y, scoring='roc_auc', n_jobs=-1, cv=5)
predictions = cross_val_predict(logreg, train_vectorized, y, cv=5)
print("Cross-Validation mean auc {0:.6f}, std {1:.2f}.".format(np.mean(scores), np.std(scores)*100))

Cross-Validation mean auc 0.999396, std 0.01.


In [0]:
logreg.fit(train_vectorized, y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
!pip install eli5
import eli5
#from eli5.sklearn import show_weights
eli5.show_weights(logreg, vec=vectorizer, targets=[0,1])

Weight?,Feature
+15.323,)
+12.148,()
+11.927,*
+11.001,""""
+9.985,\r
+9.720,\t
+9.524,"b"""
+9.391,(
+8.953,*/
+8.485,/*


In [0]:
info.head(20)

,id,text,injection
0,59290,"b""' and/**/38>( select\t(622)/**/from/*362 emh...",True
1,54992,"b'shqpkt"" union /*!426 all\t*/(select kwicwt(\...",True
2,64287,"b'nhnqag"" uniondistinct--154 298 plhlre exaloq...",True
3,28821,b'D8j+oNbylTIGw=',False
4,27825,"b'ened \\"">\\n <head>\\n <script data-reac...",False
5,74446,"b'"" union/*M!719 */'",True
6,71873,b'194 union#93 yguxsk 669 393 \r\tselect\t\tex...,True
7,87153,b'/example.example?%7Cmeta%82http-equiv=Set-Co...,True
8,76096,"b'"" /*M!union*/'",True
9,42354,b'/handler_sync_example.example?i=j4sR/mhZpekz...,False


In [0]:
eli5.show_prediction(logreg, doc=train['text'].values[3], vec=vectorizer)

In [0]:
#how well does it do against test data
score = logreg.score(test_vectorized, )